<a href="https://colab.research.google.com/github/NicolasCampana/python-seminar/blob/master/PythonSeminar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Welcome to this Notebook!
# This notebook was created to support a python seminar in May of 2020. 
# University of Modena and Reggio Emilia
# Nícolas Porto Campana 




# Part 0 - Basics and Python

---



In [0]:
# Basics
# Variables do not need to have a type
# Python interprets it
test = "T"
print(test)
test = 1
print(test)

# The if works the same way, but becareful with the tabulation! (And with the ":")
# Like C++, we can use the words "and"/"or" as keywords for &&/||
if(test == "T"):
  print("It's equal to T!")
else:
  print("Sorry, it's not equal.")


In [0]:
!pip install pulp
import requests 
import pprint
import itertools
import numpy as np
import pickle as pk
import pandas as pd
import openpyxl as pyxl
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
from pulp import *

In [0]:
# Lists and loops
x = [3, 1, 2 , 4]

print("Printing the range")
for i in range(len(x)):
  print(i)

print ("\nBut you can also use as a 'copy value'")
for i in x:
  print(i)

print ("\nIf you need the index and the value, we can use enumerate:")
for i, v in enumerate(x):
  print(f"Value {v} is in the position {i}!")

print ("\nOther very crazy thing is: python lists can hold different object types:")
x = ["Hi", 1, True, 3]

# The interpreter "transforms" the variable s to the type that is needed.
for s in x:
  print(s)

# List comprehension
crazy_list = [num for num in range(20)]
print(crazy_list)

# Transform in a list!
crazy_list = list(range(20))
print(crazy_list)


# Using the len key word
indexes = list(range(len(x)))
print(indexes)

# Code Minute
# How to access the last element of the list?

In [0]:
# You can also play with the list, cutting it
print (crazy_list[2:])

# Code minute
# How to select a interval?


In [0]:
# Dictionaries
# Very useful! Data access in O(1)!
test = {}
test['FIM'] = 10

print(test)
# Testing a Key
if ("DISMI" not in test):
  print("There is no DISMI in UNIMORE!")
  test['DISMI'] = 11
else:
  print("Okay, there is DISMI!")
print(test)

# Code Minute!
# And if we want to know if there is a value (not a key!) in the dict?

# But its possible iterating over the key and values:
for key, value in test.items():
  print(f"Key: {key}. Value: {value}")


# Part I - Web Scraping


---






In [0]:
class WebScraping(object):
  def __init__(self, url):
    self._url = url

  def request(self):
    # Code Minute!
    print("we need to do the request! But how?")    

  def execute_scraping(self) -> list:
    clients = {}
    clients_class = []
    response = self.request()
    
    if(response.status_code != 404):
      soup = BeautifulSoup(response.content, "html.parser") # Parser html 
      #print(soup.prettify()) # show the page in html
      table = soup.find('table') # Search the table in the page
      rows = table.find_all('tr') # get all the tr elements
      for idx, row in enumerate(rows[1:]): # jump the first (Header) line in the for
        client_raw = row.find_all('td') # get all the td (elements) of a row
        client_data = [e.get_text() for e in client_raw] # List comprehesion to get all the data
        print(client_data)
        #result = " - ".join(client_data) # join a separator
        #print(f"Client {idx}: {result}")
        clients[idx] = client_data
        clients_class.append(Client(*list(client_data))) # Unpack and create the clients objects
      # Code Minute!
      # how to print the name of the website?
      title = ""
      print(title)
    return clients_class
      


In [0]:
class Client(object):
  def __init__(self,name = "", time = 0, frequency = 0):
    # Code Minute!
    # And if I wanna put a id in the class?
    self._client_name = name
    self._time_to_clean = int(time)
    self._frequency = frequency

  def __repr__(self):
    return f"Client: {self._client_name}. Time to Clean: {self._time_to_clean}. Week Frequency: {self._frequency}"

In [0]:
url = "https://nicolascampana.github.io/python-seminar/"
wb = WebScraping(url)

clients = wb.execute_scraping()

for c in clients:
  print(c)

In [0]:
# And if we would want to sort the clients?
print("Sorted clients:")
new_clients = sorted(clients, key=lambda c: c._time_to_clean)
for c in new_clients:
  print(c)

# Code Minute
# But and if we sort in descending order of frequency?

# Part II - File Excel

---


In [0]:
#!wget https://github.com/NicolasCampana/python-seminar/raw/master/patterns.xlsx

def read_patterns(file:str):
  file = pd.read_excel(file)
  #print(file.head()) # Show some rows of the dataframe
  #print(file["Mon"].iloc[2:6])
  #print(file.iloc[0:5, 5:8])
  #print(file.loc[1]) # row
  matrix = file.to_numpy() # Transforms in a matrix of numpy
  #print (matrix)
  Q = {str(i):{} for i in file["Frequency"]}
  for pattern in matrix: # get each vector of the matrix
    frequency = str(pattern[0]) # get the frequency
    p_type = pattern[1] # get the type
    Q[frequency][p_type] = list(pattern[2:]) # get the real pattern 
  
  return Q


In [0]:
pat = read_patterns("patterns.xlsx")
#print(pat)
#pprint.pprint(pat)

# Part III - Assignment Model

---


In [0]:
# Using Pulp!

class SolverAssignment:    
    def __init__(self, clients:list):
        self._clients = clients
        self._D = list(range(7))
        print("Starting the assignment for the clients: ")
        #for c in self._clients:
        #  print(c)
    
    def get_client_patterns(self, patterns: dict):
        client_patterns = {}        
        for _, i in patterns.items():
            for idx, pat in i.items():
                client_patterns[idx] = pat 
        return client_patterns

    def solve_model(self, patterns:dict) -> dict:
        #Assignment Model - Pulp # https://coin-or.github.io/pulp/
        client_patterns = self.get_client_patterns(patterns)
        model = LpProblem("Assignment Problem - Pulizia", LpMinimize)
        x = LpVariable.dicts("X", [(c, p) for c in self._clients 
                                          for p in client_patterns], 
                             0, None, LpBinary)
        #x = LpVariable.dicts("X", (self.clients, client_patterns), 0, None, LpBinary)
        z = LpVariable("Z", 0, None, LpInteger)
        beta = [0, 0,  0, 0, 0, 120, 120]
        model += z, "Minimize Total balance"
        
        #print(x.keys())
        for c in self._clients:
          model += lpSum([x[(c,p)] for p in patterns[c._frequency].keys()]) == 1
        
        for d in self._D:
          model += beta[d] + lpSum([lpSum([client_patterns[p][d] * x[(c,p)] * c._time_to_clean for c in self._clients]) for p in client_patterns.keys()]) <= z

        model.solve()

        total = [0] * len(self._D) # Total assigned work in the days
        daily_demand = {i: [] for i in self._D} # daily assigned clients 
        assigned = {} # pattern choosed for each client
        patterns_choosed = [0] * (len(client_patterns)+1) # amount of times that we choose this pattern
        for c in self._clients:
            for p in client_patterns:
                if(x[(c,p)].value() > 0.6): # If the binary variable got on
                    assigned[c] = client_patterns[p]
                    patterns_choosed[p] += 1
                    for d, value in enumerate(patterns[c._frequency][p]):
                        if(value == 1):
                            daily_demand[d].append(pk.loads(pk.dumps(c))) # Copy the object using pickle
                            total[d] += c._time_to_clean 
        return daily_demand, total, assigned, patterns_choosed


In [0]:

patterns_raw = read_patterns("patterns.xlsx")
#print(patterns)
#clients_class

t = SolverAssignment(clients)
daily, total, assigned, patterns = t.solve_model(patterns_raw)
#pprint.pprint(patterns)

# Part IV - Ouput

---

In [0]:
class OutputResultsAssignment:
  def output_excel(self, daily_assignment:dict, assigned:dict, total:list, filename_assignment:str):
        print("Output - Excel")
        wb = pyxl.Workbook()
        ws1 = wb.active
        ws1.title = "Week Distributon"
        ws1.append(["Days"] + [v for v in total])
        ws1.append(["Room", "Mon","Tue","Wed","Thu","Fri","Sat", "Sun"])
       
        j = 3
        for i, k in assigned.items(): 
            ws1.cell(j, 1, f"{i._client_name}")
            for idx, day in enumerate(k):
                ws1.cell(j, idx+2, day)
            j += 1
        # Code Minute
        # How to do it using append?
        wb.save(filename_assignment)
        

In [0]:
output_file = "OutputClients.xlsx"
OutputResultsAssignment().output_excel(daily, assigned, total, output_file)

# Part V - Analyse
---

In [0]:
file = pd.read_excel(output_file, skiprows=1)

days = list(file.keys()[1:])
print (days)

# Which patterns are choosed by each client
for i, k in assigned.items():
  all_visits = [visits for visits in list(itertools.compress(days,k))]
  print(f"Client {i._client_name}, choosed the pattern: {k}")
  print(f"It means that we will visit in the days: " + " - ".join(all_visits))

In [0]:
# Total work
for i, k in enumerate(total):
  print(f"In {days[i]}, we have a total of {k} minutes of work.")

# Code Minute
# And if we want to know the sum of all work?

In [0]:
# See seaborn
# See plotly
# The total amount of work in time
data = pd.Series(total) # Transforms in a serie variable
data.plot()
print(f"The mean time for each day is {data.mean()}")
print(f"The standard deviation is {data.std()}")
print(f"The day that we work more is {data.max()}, and the less {data.min()}")

In [0]:
# The total amount of clients for each day
data = pd.Series([len(k) for _, k in daily.items()])
#print(data.describe())
data.plot()
print(f"The mean value is {int(data.mean())}")
print(f"The standard deviation is {int(data.std())}")
print(f"The day that we work more is {data.max()}, and the less {data.min()}")

In [0]:
# The frequency of patterns
data = pd.Series(patterns)
data.plot()
print(f"The most choosed pattern have {data.max()} times, and the less {data.min()} times.")
print(f"And the pattern most choosed pattern is {data.argmax()}!")


In [0]:
# How to calculate the last demand from the company?
# How much would it cost for the company?
hour_cost = 10.0